<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and running the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
    
</div>

#  Lorentzian + background with lmfit

## Table of Contents

- [Introduction](#Introduction)
- [Importing the required libraries](#Importing-the-required-libraries)
- [Plot of the fitting model](#Plot-of-the-fitting-model)
- [Creating the reference data](#Creating-the-reference-data)
- [Setting and fitting](#Setting-and-fitting)  
- [Plotting the results](#Plotting-the-results) 

[Top](#Table-of-Contents)

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to combine the models of 
the <a href="https://github.com/QENSlibrary/QENSmodels">QENSlibrary</a>. Here, we use the <b>Lorentzian</b> profile and a flat background, created from <b>background_polynomials</b>, to perform some fits.

<a href="https://lmfit.github.io/lmfit-py/">lmfit</a> is used for fitting.
</div>

[Top](#Table-of-Contents)

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import ipywidgets

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = ipywidgets.Button(description='Yes', button_style='success')
    buttonN = ipywidgets.Button(description='No', button_style='danger')
    choice_installation = ipywidgets.VBox(
    [ipywidgets.Label("Do you want to install the QENSmodels' library?"), ipywidgets.HBox([buttonY, buttonN])],
    layout=ipywidgets.Layout(width='50%', height='80px'))
    
    display(choice_installation)
    
    def on_buttonY_clicked(b):
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels

    def on_buttonN_clicked(b):
        print("You will not be able to run some of the remaining parts of this notebook")
     
    buttonY.on_click(on_buttonY_clicked)
    buttonN.on_click(on_buttonN_clicked) 

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = ipywidgets.Button(description='Yes', button_style='success')
    lmfitN = ipywidgets.Button(description='No', button_style='danger')
    choice_installation = ipywidgets.VBox(
    [ipywidgets.Label("Do you want to install lmfit?"), ipywidgets.HBox([lmfitY, lmfitN])],
    layout=ipywidgets.Layout(width='30%', height='80px'))

    display(choice_installation)
    
    def on_lmfitY_clicked(b):
        !{sys.executable} -m pip install lmfit
    def on_lmfitN_clicked(b):
        print("You will not be able to run some of the remaining parts of this notebook")  
    lmfitY.on_click(on_lmfitY_clicked)
    lmfitN.on_click(on_lmfitN_clicked) 

In [ ]:
# required imports from lmfit
from lmfit import Model

# import model from QENS library
import QENSmodels

### Physical units

For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "1/ps",
                       'scale': "unit_of_signal/ps",
                       'center': "1/ps",
                       'hwhm': "1/ps"}

[Top](#Table-of-Contents)

## Plot of the fitting model

The widget below shows the lorentzian peak shape function with a constant background imported from QENSmodels where the functions' parameters *Scale*, *Center*, *FWHM* and *background* can be varied.

In [ ]:
# Dictionary of initial values
ini_parameters = {'scale': 5, 'center': 0, 'hwhm': 3, 'background': 0.}

def interactive_fct(scale, center, hwhm, background):
    xs = np.linspace(-10, 10, 100)
    
    fig1, ax1 = plt.subplots()
    ax1.plot(xs, QENSmodels.lorentzian(xs, scale, center, hwhm) +\
             QENSmodels.background_polynomials(xs, background))
    ax1.set_xlabel('x')
    ax1.grid()

# Define sliders for modifiable parameters and their range of variations

scale_slider = ipywidgets.FloatSlider(value=ini_parameters['scale'],
                                      min=0.1, max=10, step=0.1,
                                      description='scale',
                                      continuous_update=False) 

center_slider = ipywidgets.IntSlider(value=ini_parameters['center'],
                                     min=-10, max=10, step=1,
                                     description='center', 
                                     continuous_update=False) 

hwhm_slider = ipywidgets.FloatSlider(value=ini_parameters['hwhm'],
                                     min=0.1, max=10, step=0.1,
                                     description='hwhm',
                                     continuous_update=False)

background_slider = ipywidgets.FloatSlider(value=ini_parameters['background'],
                                       min=0.1, max=10, step=0.1,
                                       description='background',
                                       continuous_update=False)

grid_sliders = ipywidgets.HBox([ipywidgets.VBox([scale_slider, center_slider])
                                ,ipywidgets.VBox([hwhm_slider, background_slider])])
                               
# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    scale_slider.value = ini_parameters['scale'] 
    center_slider.value = ini_parameters['center']  
    hwhm_slider.value = ini_parameters['hwhm'] 
    background_slider.value = ini_parameters['background'] 


# Define reset button and occurring action when clicking on it
reset_button = ipywidgets.Button(description = "Reset")
reset_button.on_click(reset_values)

# Display the interactive plot
interactive_plot = ipywidgets.interactive_output(interactive_fct,       
                                         {'scale': scale_slider,
                                          'center': center_slider,
                                          'hwhm': hwhm_slider,
                                          'background': background_slider})  
                                            
display(grid_sliders, interactive_plot, reset_button)

[Top](#Table-of-Contents)

## Creating the reference data

In [ ]:
# Create array of reference data: noisy lorentzian with background
nb_points = 100
xx = np.linspace(-5, 5, nb_points)
added_noise = 0.02 * np.random.normal(0, 1, nb_points)
lorentzian_noisy = QENSmodels.lorentzian(xx,
                                         scale=0.89,
                                         center=-0.025,
                                         hwhm=0.45) * (1 + 5 * added_noise)
lorentzian_noisy += 0.5 * (1. + added_noise)

[Top](#Table-of-Contents)

## Setting and fitting

In [ ]:
def flat_background(x, A0):
    """ Define flat background ot be added to fitting model"""
    return QENSmodels.background_polynomials(x, A0)

In [ ]:
gmodel = Model(QENSmodels.lorentzian) + Model(flat_background)
print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1, 0.2, 0.5, 0.33]

# Fit
result = gmodel.fit(lorentzian_noisy,
                    x=xx,
                    scale=initial_parameters_values[0],
                    center=initial_parameters_values[1],
                    hwhm=initial_parameters_values[2],
                    A0=initial_parameters_values[3])

In [ ]:
# Plot initial model and reference data
fig0, ax0 = plt.subplots()
ax0.plot(xx, lorentzian_noisy, 'b-', label='reference data')
ax0.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax0.set(xlabel='x', title='Initial model and reference data')
ax0.grid()
ax0.legend();

[Top](#Table-of-Contents)

## Plotting results

using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n', result.fit_report())

In [ ]:
# plot fitting result using lmfit functionality
result.plot()

In [ ]:
# Plot fitting result and reference data using matplotlib.pyplot
fig1, ax1 = plt.subplots()
ax1.plot(xx, lorentzian_noisy, 'b-', label='reference data')
ax1.plot(xx, result.best_fit, 'r.', label='fitting result')
ax1.legend()
ax1.set(xlabel='x', title='Fit result and reference data')
ax1.grid();

In [ ]:
for item in ['hwhm', 'center', 'scale']:
    print(item,
          result.params[item].value, '+/-', result.params[item].stderr, dict_physical_units[item])